#####Libraries
***

In [1]:
import sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
# pip install TA-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib
import talib as ta
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 16.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
--2022-09-16 13:57:53--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-16 13:57:54--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownl

#####Load Dataset
***

In [2]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
  #άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-05-29','2022-09-25')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο και 3 μήνες
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-05-29','2022-09-25')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [3]:
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,106.605820,81.597748,158.413422,136.604294,136.802216,256.874268,160.932495,203.371292,218.499527,161.456619,...,102.610245,259.269989,252.285995,33.223030,120.749809,398.641052,339.450012,224.728790,169.534714,56.981377
2021-06-02,105.836273,82.125130,156.835663,135.604462,137.594040,257.275818,161.699493,205.369492,215.377518,161.811584,...,102.156342,262.170013,249.654388,33.380482,118.805557,397.788361,338.470001,227.733353,171.020294,57.433758
2021-06-03,106.605820,82.422363,159.030807,135.212387,134.990860,258.382538,159.350494,205.707352,215.249908,162.205963,...,102.345467,257.790009,249.566986,32.563683,117.322830,399.856445,336.119995,226.196381,170.178467,57.659946
2021-06-04,106.748322,82.489479,159.834412,136.084778,138.425491,259.939758,160.311005,204.404190,216.300385,162.679214,...,102.544044,263.040009,251.868439,32.662086,117.313004,397.582520,332.070007,228.209320,170.990570,57.914417
2021-06-07,107.365860,82.144295,159.295410,135.271164,137.554459,259.401062,159.900497,204.780655,212.471527,163.586243,...,102.194168,260.600006,250.334167,32.278294,117.347458,392.299591,316.019989,229.379425,166.187180,57.537434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,142.240005,105.180000,163.490005,100.773888,96.300003,289.239990,136.449997,214.220001,192.050003,162.449997,...,99.760002,97.650002,282.829987,36.259998,94.980003,531.250000,313.399994,206.630005,192.509048,97.610001
2022-09-13,138.529999,102.199997,158.490005,95.476807,90.389999,277.720001,126.820000,207.850006,184.669998,155.350006,...,95.790001,95.010002,279.480011,35.930000,89.860001,513.960022,294.670013,199.669998,185.940002,95.330002
2022-09-14,139.550003,103.320000,155.919998,94.739998,90.639999,277.959991,128.550003,205.080002,180.190002,156.300003,...,94.080002,97.660004,280.149994,36.330002,89.459999,509.769989,290.000000,199.410004,185.910004,97.669998


In [4]:
rd1= rd.pct_change()
rd1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,-0.007219,0.006463,-0.009960,-0.007319,0.005788,0.001563,0.004766,0.009825,-0.014288,0.002199,...,-0.004424,0.011185,-0.010431,0.004739,-0.016101,-0.002139,-0.002887,0.013370,0.008763,0.007939
2021-06-03,0.007271,0.003619,0.013996,-0.002891,-0.018919,0.004302,-0.014527,0.001645,-0.000592,0.002437,...,0.001851,-0.016707,-0.000350,-0.024469,-0.012480,0.005199,-0.006943,-0.006749,-0.004922,0.003938
2021-06-04,0.001337,0.000814,0.005053,0.006452,0.025443,0.006027,0.006028,-0.006335,0.004880,0.002918,...,0.001940,0.020365,0.009222,0.003022,-0.000084,-0.005687,-0.012049,0.008899,0.004772,0.004413
2021-06-07,0.005785,-0.004185,-0.003372,-0.005979,-0.006292,-0.002072,-0.002561,0.001842,-0.017702,0.005576,...,-0.003412,-0.009276,-0.006092,-0.011750,0.000294,-0.013288,-0.048333,0.005127,-0.028092,-0.006509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,0.005798,0.004489,-0.001283,0.015859,-0.002176,0.009176,0.023861,-0.001119,0.005234,0.025309,...,0.012998,0.014756,0.000495,0.003876,0.015720,0.013178,0.007264,0.006969,0.004114,0.011503
2022-09-13,-0.026083,-0.028332,-0.030583,-0.052564,-0.061371,-0.039828,-0.070575,-0.029736,-0.038428,-0.043706,...,-0.039796,-0.027035,-0.011844,-0.009101,-0.053906,-0.032546,-0.059764,-0.033683,-0.034123,-0.023358
2022-09-14,0.007363,0.010959,-0.016216,-0.007717,0.002766,0.000864,0.013641,-0.013327,-0.024259,0.006115,...,-0.017852,0.027892,0.002397,0.011133,-0.004451,-0.008152,-0.015848,-0.001302,-0.000161,0.024546


In [5]:
#prepare the ranks for next month (22 working days later)
rd1_next_month = rd1.shift(-22)
rd1_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,0.014382,0.007920,0.004290,0.006533,-0.031250,0.008478,-0.002090,0.008721,0.000143,0.012985,...,0.008100,-0.008508,0.019087,0.011988,0.007984,0.011113,0.007335,0.005688,0.011905,0.002853
2021-06-02,0.007964,-0.000704,-0.002232,-0.001213,0.001522,-0.000279,0.022724,0.003895,-0.001760,0.009345,...,-0.001646,0.004291,-0.017405,0.001445,0.010081,0.011040,-0.005539,0.014799,0.003959,-0.001423
2021-06-03,0.004862,0.000470,0.001981,-0.010568,-0.009988,-0.014865,0.046927,0.010505,-0.016103,0.006291,...,-0.028992,0.008269,-0.015732,0.008367,0.002139,0.002199,-0.025472,0.004065,0.000620,-0.028495
2021-06-04,0.008814,0.004458,0.001020,0.010537,-0.013964,0.007039,0.005670,0.010208,0.014478,0.008375,...,0.002596,0.015343,0.010468,0.018026,-0.019206,0.003412,0.004367,0.001669,0.025224,-0.015643
2021-06-07,-0.003426,-0.003621,-0.016691,-0.017855,-0.017350,-0.030531,0.009422,0.007139,-0.009689,-0.020353,...,-0.032370,-0.007000,-0.031835,-0.016020,-0.019502,-0.004154,-0.021005,-0.014125,-0.004833,-0.004469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#first: ranks assigned in order they appear in the array
rd2=rd1_next_month.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,39,22,16,18,1,26,7,29,9,38,...,24,2,42,36,23,33,21,17,35,13
2021-06-02,35,14,6,12,19,16,45,23,8,37,...,9,25,1,18,39,40,3,42,24,10
2021-06-03,37,28,32,15,16,12,45,43,9,39,...,4,41,10,42,33,34,6,36,30,5
2021-06-04,28,18,9,33,5,22,20,30,38,27,...,13,39,32,43,3,16,17,11,45,4
2021-06-07,35,34,12,9,10,3,44,43,22,6,...,1,27,2,13,8,32,5,16,30,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-10,22,19,5,39,4,27,41,7,20,42,...,34,37,10,17,38,35,24,23,18,30
2022-08-11,35,30,27,8,4,13,3,29,17,11,...,14,33,44,45,7,23,5,21,20,37
2022-08-12,34,36,7,17,29,26,40,10,4,32,...,5,44,28,37,20,15,8,24,25,43


In [7]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,5,3,2,2,1,3,1,4,1,5,...,3,1,5,4,3,4,3,2,4,2
2021-06-02,4,2,1,2,3,2,5,3,1,5,...,1,3,1,2,5,5,1,5,3,2
2021-06-03,5,4,4,2,2,2,5,5,1,5,...,1,5,2,5,4,4,1,4,4,1
2021-06-04,4,2,1,4,1,3,3,4,5,3,...,2,5,4,5,1,2,2,2,5,1
2021-06-07,4,4,2,1,2,1,5,5,3,1,...,1,3,1,2,1,4,1,2,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-10,3,3,1,5,1,3,5,1,3,5,...,4,5,2,2,5,4,3,3,2,4
2022-08-11,4,4,3,1,1,2,1,4,2,2,...,2,4,5,5,1,3,1,3,3,5
2022-08-12,4,4,1,2,4,3,5,2,1,4,...,1,5,4,5,3,2,1,3,3,5


In [8]:
#for absolute rank feature to use 
rd2_feature=rd1.rank(axis=1,method='first')
rd2_feature=rd2_feature.dropna()
rd2_feature.astype(int)
jk_feature=rd2_feature.copy()
jk_feature.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-02,10,32,5,9,31,24,30,38,3,25,...,12,39,4,29,2,15,14,41,36,35
2021-06-03,37,33,41,15,2,35,4,24,19,30,...,27,3,20,1,5,36,9,11,13,34
2021-06-04,15,13,29,34,45,30,31,4,28,20,...,17,44,36,21,10,5,2,35,27,26
2021-06-07,38,20,23,17,15,28,26,32,3,37,...,22,12,16,8,31,7,1,36,2,14
2021-06-08,8,1,28,32,2,13,44,9,33,23,...,19,36,27,35,34,24,45,22,38,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,22,19,5,39,4,27,41,7,20,42,...,34,37,10,17,38,35,24,23,18,30
2022-09-13,35,30,27,8,4,13,3,29,17,11,...,14,33,44,45,7,23,5,21,20,37
2022-09-14,34,36,7,17,29,26,40,10,4,32,...,5,44,28,37,20,15,8,24,25,43


In [9]:
r5_feature=rd2_feature.copy()
rd3_feature=rank_data(r5_feature)
rd3_feature

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-02,2,4,1,1,4,3,4,5,1,3,...,2,5,1,4,1,2,2,5,4,4
2021-06-03,5,4,5,2,1,4,1,3,3,4,...,3,1,3,1,1,4,1,2,2,4
2021-06-04,2,2,4,4,5,4,4,1,4,3,...,2,5,4,3,2,1,1,4,3,3
2021-06-07,5,3,3,2,2,4,3,4,1,5,...,3,2,2,1,4,1,1,4,1,2
2021-06-08,1,1,4,4,1,2,5,1,4,3,...,3,4,3,4,4,3,5,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,3,3,1,5,1,3,5,1,3,5,...,4,5,2,2,5,4,3,3,2,4
2022-09-13,4,4,3,1,1,2,1,4,2,2,...,2,4,5,5,1,3,1,3,3,5
2022-09-14,4,4,1,2,4,3,5,2,1,4,...,1,5,4,5,3,2,1,3,3,5


In [10]:
rd3.loc[rd3.index=='2022-08-19']

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,


#####Functions
***

In [11]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    # max_date = np.max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [12]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [53]:
def predictions(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      # print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-79] #-44  -79
      
      Y_train= y.iloc[:-79] #-44  -79

      print(Y_train.tail(2))
      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      flag=True
      for i in range(58,0,-1): #from 4/01 and later -->predictions 23   58
        if(flag):
          first=X.iloc[-(i+1):-i].index
          print('this is the first day of predictions', first)
          flag=False
        X_validation=X.iloc[-(i+1):-i] 
        # print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        # print(Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      X_validation=X.tail(1) 
      last=X_validation.index
      print('this is last for prediction',last)
      # print(X_validation)
      Y_validation= y.tail(1)
      print(Y_validation.index) 
      probability= model.predict_proba(X_validation)
      print(probability[0])
      tmp.append(probability[0])
      prob[j]=tmp
  return prob , first , last 

In [14]:
def RPS_data(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob : ',prob)
      mask1 = (rd.loc[(rd.index.date>=prob_final['date'][k+1])].index)
      prob1_table = rd.loc[mask1]
      prob1 = prob1_table.shift(-21).head(1)
      print('prob1 : ',prob1)

      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob teleutaio :', prob)
      prob1 = pd.DataFrame(prob_table.iloc[1]).T
      print('prob1 : ',prob1)
      
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [15]:
def create_RPS(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

####**For 29 May-24 June** 

#####Features
***

In [16]:
stock=dict()
for i in assets:
  stock[i]=pdr.get_data_yahoo(i,'2021-05-01','2022-06-25') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-05-27') & (stock['ABBV'].index <= '2022-06-24')]))
# stock['ABBV']

18


In [17]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-05-03']['Rank'])

Date
2022-05-03    2.0
Name: Rank, dtype: float64
Date
2022-05-03    2.0
Name: Rank, dtype: float64
Date
2022-05-03    1.0
Name: Rank, dtype: float64
Date
2022-05-03    2.0
Name: Rank, dtype: float64
Date
2022-05-03    2.0
Name: Rank, dtype: float64
Date
2022-05-03    3.0
Name: Rank, dtype: float64
Date
2022-05-03    5.0
Name: Rank, dtype: float64
Date
2022-05-03    2.0
Name: Rank, dtype: float64
Date
2022-05-03    4.0
Name: Rank, dtype: float64
Date
2022-05-03    4.0
Name: Rank, dtype: float64
Date
2022-05-03    4.0
Name: Rank, dtype: float64
Date
2022-05-03    3.0
Name: Rank, dtype: float64
Date
2022-05-03    3.0
Name: Rank, dtype: float64
Date
2022-05-03    4.0
Name: Rank, dtype: float64
Date
2022-05-03    1.0
Name: Rank, dtype: float64
Date
2022-05-03    3.0
Name: Rank, dtype: float64
Date
2022-05-03    4.0
Name: Rank, dtype: float64
Date
2022-05-03    1.0
Name: Rank, dtype: float64
Date
2022-05-03    2.0
Name: Rank, dtype: float64
Date
2022-05-03    2.0
Name: Rank, dtype: float64


#####Gradient Boosting (baseline)
***

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-04-20    2.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.18086979 0.06534057 0.14668919 0.5343455  0.07275495]
Date
2022-04-20    4.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.11418182 0.08578109 0.47263889 0.03004113 0.29735708]
Date
2022-04-20    1.0
2022-04-21    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-23  148.065018  91.689781  168.615479  96.392845  93.042061   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-23  238.452896  112.440002  194.904602  154.861267  141.121872  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-23  91.416611  73.809998  270.594604  34.059608  73.720894   

                   UNH         URI           V       VRSK        XOM  
Date                                                                  
2022-05-23  498.246613  235.839996  196.292328  170.68544  84.390045  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-24  150.928238  92.870499  172.150925  99.170822  97.111656   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-24  248.137924  116.459999  193.575409  164.681396  145.605087  ...   

                  PRU   PYPL         RE        ROL       ROST         UNH  \
Date                                                                        
2022-05-24  96.279518  77.68  281.48877  35.017067  77.258453  494.089691   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-24  248.289993  205.146637  172.702087  86.063782  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                      

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-06-23  150.220001  95.05336  171.254623  102.566132  100.742371   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-23  244.902954  121.139999  197.539993  174.073257  153.789993  ...   

                  PRU       PYPL        RE        ROL       ROST         UNH  \
Date                                                                           
2022-06-23  95.745781  81.650002  272.1651  36.393417  82.340591  527.813843   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-23  277.829987  213.891159  182.465805  89.114151  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-24  150.869995  95.668518  171.234695  102.247513  98.657707   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-24  240.463562  114.809998  197.539993  171.065475  149.830002  ...   

                  PRU       PYPL          RE       ROL       ROST         UNH  \
Date                                                                            
2022-06-24  94.935303  77.040001  270.187073  36.12413  77.686951  529.927246   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-24  276.209991  212.114319  182.705399  88.767509  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
20

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20,0.126559,0.196758,0.181878,0.135055,0.252405,0.151357,0.279727,0.128245,0.177283,0.405772,...,0.090901,0.397890,0.099581,0.401993,0.162407,0.065834,0.191384,0.083032,0.221117,0.463979
2022-05-23,0.150601,0.350518,0.103069,0.099928,0.338384,0.046084,0.234649,0.435814,0.578193,0.103274,...,0.334294,0.673054,0.091801,0.244632,0.027471,0.412150,0.503972,0.089908,0.586876,0.267938
2022-05-24,0.489418,0.235238,0.101305,0.456426,0.133485,0.337429,0.039416,0.206888,0.163088,0.385293,...,0.070868,0.000070,0.038695,0.112923,0.464782,0.082924,0.425117,0.142639,0.000496,0.179771
2022-05-25,0.181273,0.220559,0.325230,0.002404,0.140478,0.123312,0.380864,0.014681,0.083190,0.216884,...,0.403554,0.105152,0.100339,0.237283,0.158836,0.266359,0.147967,0.058426,0.588118,0.131786
2022-05-26,0.287615,0.041049,0.344926,0.108868,0.078487,0.130356,0.483554,0.077946,0.150162,0.743344,...,0.158504,0.376102,0.189506,0.256708,0.138806,0.144003,0.198948,0.195112,0.182740,0.181568
2022-05-27,0.043158,0.210535,0.104368,0.087819,0.000294,0.370090,0.010925,0.064635,0.099989,0.198718,...,0.053345,0.082814,0.050557,0.025397,0.043682,0.109220,0.516393,0.131618,0.189971,0.285231
2022-05-31,0.053197,0.001778,0.025332,0.030022,0.345412,0.480916,0.641046,0.086212,0.124795,0.049432,...,0.109138,0.659556,0.032760,0.088800,0.545663,0.184643,0.309103,0.191071,0.117449,0.647062
2022-06-01,0.192734,0.357973,0.129798,0.148135,0.098646,0.097885,0.435683,0.165745,0.017688,0.028194,...,0.276705,0.654948,0.464938,0.075300,0.011601,0.294434,0.165999,0.184935,0.141959,0.141463
2022-06-02,0.322353,0.062772,0.415461,0.042087,0.124738,0.112481,0.242244,0.276391,0.329459,0.241259,...,0.149401,0.094312,0.019330,0.334663,0.115294,0.250536,0.242081,0.222769,0.002151,0.399003


In [19]:
print(RPS1.mean(axis=1).mean())

0.2140680599090975


In [20]:
print(RPS1.mean(axis=1))

date
2022-05-20    0.195966
2022-05-23    0.228231
2022-05-24    0.199232
2022-05-25    0.181584
2022-05-26    0.194483
2022-05-27    0.192152
2022-05-31    0.238419
2022-06-01    0.236266
2022-06-02    0.183697
2022-06-03    0.263916
2022-06-06    0.199230
2022-06-07    0.202091
2022-06-08    0.262426
2022-06-09    0.189110
2022-06-10    0.217466
2022-06-13    0.248132
2022-06-14    0.251845
2022-06-15    0.217053
2022-06-16    0.227483
2022-06-17    0.250488
2022-06-21    0.190803
2022-06-22    0.189371
2022-06-23    0.154837
2022-06-24    0.223351
dtype: float64


In [23]:
#save results 
RPS1.to_excel('29_May_24_June-gradient(baseline)-one_m_less.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('29_May_24_June-PROBABILITIES-gradient(baseline)-one_m_less.xlsx')

#####After feature importances 
***

In [24]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [25]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-04-20    2.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15539705 0.18586041 0.19600809 0.22286236 0.2398721 ]
Date
2022-04-20    4.0
2022-04-21    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15397807 0.2188721  0.17742981 0.21390197 0.23581806]
Date
2022-04-20    1.0
2022-04-21    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-23  148.065018  91.689781  168.615479  96.392845  93.042061   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-23  238.452896  112.440002  194.904602  154.861267  141.121872  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-23  91.416611  73.809998  270.594604  34.059608  73.720894   

                   UNH         URI           V       VRSK        XOM  
Date                                                                  
2022-05-23  498.246613  235.839996  196.292328  170.68544  84.390045  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-24  150.928238  92.870499  172.150925  99.170822  97.111656   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-24  248.137924  116.459999  193.575409  164.681396  145.605087  ...   

                  PRU   PYPL         RE        ROL       ROST         UNH  \
Date                                                                        
2022-05-24  96.279518  77.68  281.48877  35.017067  77.258453  494.089691   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-24  248.289993  205.146637  172.702087  86.063782  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                      

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-06-23  150.220001  95.05336  171.254623  102.566132  100.742371   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-23  244.902954  121.139999  197.539993  174.073257  153.789993  ...   

                  PRU       PYPL        RE        ROL       ROST         UNH  \
Date                                                                           
2022-06-23  95.745781  81.650002  272.1651  36.393417  82.340591  527.813843   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-23  277.829987  213.891159  182.465805  89.114151  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-24  150.869995  95.668518  171.234695  102.247513  98.657707   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-24  240.463562  114.809998  197.539993  171.065475  149.830002  ...   

                  PRU       PYPL          RE       ROL       ROST         UNH  \
Date                                                                            
2022-06-24  94.935303  77.040001  270.187073  36.12413  77.686951  529.927246   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-24  276.209991  212.114319  182.705399  88.767509  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
20

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20,0.082090,0.234836,0.288015,0.073366,0.141492,0.140396,0.272903,0.248774,0.102627,0.239676,...,0.119723,0.134220,0.132870,0.223084,0.133707,0.104275,0.203456,0.082618,0.237383,0.268153
2022-05-23,0.145888,0.257800,0.122647,0.073366,0.141031,0.096749,0.155699,0.247211,0.277274,0.075032,...,0.241876,0.288012,0.112466,0.084271,0.110239,0.282269,0.235491,0.124202,0.278430,0.161563
2022-05-24,0.106848,0.232682,0.058617,0.117347,0.119037,0.140396,0.192590,0.248774,0.128131,0.242979,...,0.119723,0.145563,0.073129,0.088485,0.222614,0.218737,0.234291,0.223019,0.098447,0.169794
2022-05-25,0.097405,0.232422,0.252522,0.117032,0.141492,0.073428,0.192590,0.113302,0.102627,0.075032,...,0.241876,0.145563,0.112466,0.135854,0.222614,0.218737,0.107385,0.114665,0.278430,0.184031
2022-05-26,0.097360,0.104180,0.123947,0.120559,0.217702,0.275864,0.272903,0.074023,0.128131,0.243578,...,0.119723,0.134220,0.132870,0.217542,0.109844,0.218737,0.210293,0.128589,0.249244,0.282164
2022-05-27,0.082796,0.229447,0.244083,0.071959,0.145761,0.140396,0.194688,0.074023,0.102627,0.130451,...,0.072077,0.145822,0.214121,0.109359,0.212207,0.067467,0.238115,0.112167,0.250949,0.268153
2022-05-31,0.149221,0.106640,0.055736,0.124054,0.229521,0.275864,0.281578,0.114910,0.066844,0.074015,...,0.119198,0.303639,0.073129,0.253787,0.135040,0.136947,0.129065,0.080453,0.054914,0.184031
2022-06-01,0.095643,0.262442,0.126300,0.120357,0.116581,0.140396,0.281578,0.118768,0.066844,0.074015,...,0.255185,0.302987,0.275022,0.088485,0.254481,0.282269,0.138526,0.128589,0.112956,0.268153
2022-06-02,0.283866,0.229447,0.105577,0.071959,0.141031,0.140396,0.163367,0.118768,0.277274,0.237535,...,0.125036,0.145563,0.214121,0.217542,0.111068,0.218737,0.211952,0.080607,0.248117,0.268153


In [26]:
print(RPS2.mean(axis=1).mean())

0.15921967308025917


In [27]:
print(RPS2.mean(axis=1))

date
2022-05-20    0.162312
2022-05-23    0.166676
2022-05-24    0.150158
2022-05-25    0.151973
2022-05-26    0.159072
2022-05-27    0.154807
2022-05-31    0.161058
2022-06-01    0.169078
2022-06-02    0.155554
2022-06-03    0.161010
2022-06-06    0.152746
2022-06-07    0.151930
2022-06-08    0.160789
2022-06-09    0.159704
2022-06-10    0.163005
2022-06-13    0.164619
2022-06-14    0.162808
2022-06-15    0.164228
2022-06-16    0.165580
2022-06-17    0.168045
2022-06-21    0.150313
2022-06-22    0.151287
2022-06-23    0.152282
2022-06-24    0.162236
dtype: float64


In [28]:

#save results 
RPS2.to_excel('29_May_24_June-gradient(with_feature_imp)-one_m_less.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('29_May_24_June-PROBABILITIES-gradient(with_feature_imp)-one_m_less.xlsx')

####**For 26 June.-22 July.**

#####Features
***

In [29]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-06-01','2022-07-23') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-06-01  113.699997  113.989998  112.040001  112.209999  106.605812
 2021-06-02  112.410004  112.660004  110.849998  111.400002  105.836273
 2021-06-03  111.120003  112.410004  110.650002  112.209999  106.605812
 2021-06-04  112.400002  113.110001  112.160004  112.360001  106.748337
 2021-06-07  112.709999  114.110001  112.330002  113.010002  107.365868
 ...                ...         ...         ...         ...         ...
 2022-07-18  153.800003  154.470001  148.889999  149.570007  149.570007
 2022-07-19  150.000000  150.229996  148.800003  149.740005  149.740005
 2022-07-20  149.529999  150.449997  147.339996  147.690002  147.690002
 2022-07-21  147.240005  147.839996  146.020004  147.750000  147.750000
 2022-07-22  148.470001  149.289993  147.580002  148.470001  148.470001
 
 [289 rows x 5 columns],
 'AEP':                  Open

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-06-26') & (stock['ABBV'].index <= '2022-07-22')]))

19


In [30]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-05-26']['Rank'])

Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    2
Name: Rank, dtype: int64
Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    1
Name: Rank, dtype: int64
Date
2022-05-26    1
Name: Rank, dtype: int64
Date
2022-05-26    5
Name: Rank, dtype: int64
Date
2022-05-26    3
Name: Rank, dtype: int64
Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    1
Name: Rank, dtype: int64
Date
2022-05-26    5
Name: Rank, dtype: int64
Date
2022-05-26    3
Name: Rank, dtype: int64
Date
2022-05-26    3
Name: Rank, dtype: int64
Date
2022-05-26    5
Name: Rank, dtype: int64
Date
2022-05-26    2
Name: Rank, dtype: int64
Date
2022-05-26    2
Name: Rank, dtype: int64
Date
2022-05-26    5
Name: Rank, dtype: int64
Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    1
Name: Rank, dtype: int64
Date
2022-05-26    4
Name: Rank, dtype: int64
Date
2022-05-26    1
Name: Rank, d

#####Gradient Boosting (baseline)
***

In [31]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-05-17    2
2022-05-18    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.78435631 0.05410278 0.05196133 0.0999546  0.00962499]
Date
2022-05-17    1
2022-05-18    3
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.03552439 0.08047972 0.68960251 0.10591246 0.08848091]
Date
2022-05-17    4
2022-05-18    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                ABBV        AEP         AIZ        ALLE        AMAT         AMP  \
Date                                                                            
2022-06-21  147.75  92.066818  168.087662  103.014191  103.575127  244.206192   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-06-21  124.629997  196.050003  175.318207  150.179993  ...  94.955063   

             PYPL          RE       ROL       ROST         UNH         URI  \
Date                                                                         
2022-06-21  82.75  268.616547  36.46323  83.317162  520.825806  279.790009   

                    V        VRSK        XOM  
Date                                          
2022-06-21  215.80777  184.402573  86.905602  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-22  148.470001  93.614655  169.043716  102.944489  101.41066   

                   AMP        AMZN     AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-06-22  243.927475  122.419998  198.25  174.591156  153.009995  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-06-22  94.401566  81.050003  270.306335  36.373466  82.948456   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-06-22  519.779053  274.290009  213.322159  184.472458  86.24205  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                 

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-21  141.850006  104.940002  171.145065  105.991302  104.363106   

                  AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                  ...   
2022-07-21  284.73999  138.229996  214.979996  196.25322  162.240005  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-07-21  102.199997  96.559998  279.808899  36.540001  90.691246   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-07-21  546.604919  316.690002  212.820007  200.236176  94.080002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
D

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-22  140.339996  103.410004  164.163818  102.695572  100.552856   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-07-22  276.98999  133.220001  210.330002  193.584045  157.699997  ...   

                  PRU       PYPL         RE        ROL       ROST         UNH  \
Date                                                                            
2022-07-22  99.110001  93.580002  273.01001  36.400002  87.801399  542.866638   

                   URI           V        VRSK        XOM  
Date                                                       
2022-07-22  307.549988  208.610001  197.900055  94.010002  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-06-17,0.169716,0.190501,0.370612,0.556110,0.255292,0.271829,0.088400,0.232661,0.322256,0.098750,...,0.049609,0.036693,0.180481,0.200759,0.122194,0.171807,0.139289,0.082682,0.127819,0.137538
2022-06-21,0.344186,0.401685,0.267813,0.116371,0.263790,0.325391,0.537642,0.527613,0.094497,0.357448,...,0.145123,0.547118,0.320329,0.226799,0.132241,0.098231,0.389698,0.171170,0.026857,0.180945
2022-06-22,0.123493,0.311646,0.267643,0.102762,0.001482,0.251478,0.191332,0.196278,0.067008,0.248948,...,0.202316,0.126347,0.163978,0.203825,0.027484,0.078396,0.168082,0.094154,0.602319,0.189143
2022-06-23,0.196941,0.394602,0.553353,0.106388,0.020406,0.000881,0.715303,0.279896,0.000299,0.252232,...,0.165252,0.349221,0.056074,0.020623,0.173966,0.225896,0.007218,0.116465,0.144883,0.114453
2022-06-24,0.163952,0.124775,0.177631,0.168866,0.390477,0.592749,0.171407,0.306475,0.529048,0.173759,...,0.039225,0.140170,0.338634,0.178431,0.115759,0.092002,0.378734,0.355092,0.200091,0.201013
2022-06-27,0.016502,0.257541,0.193368,0.043899,0.193720,0.143338,0.075487,0.342479,0.220944,0.273895,...,0.004639,0.011580,0.225859,0.329274,0.221151,0.099964,0.482382,0.086951,0.094543,0.122741
2022-06-28,0.650791,0.131480,0.183852,0.237535,0.134456,0.384123,0.124216,0.075603,0.145270,0.135044,...,0.144169,0.054444,0.081553,0.381856,0.080057,0.090292,0.765338,0.106751,0.052432,0.027247
2022-06-29,0.247542,0.261509,0.116765,0.119244,0.571289,0.194975,0.089482,0.198456,0.354153,0.101695,...,0.153675,0.738415,0.284548,0.368609,0.171060,0.557033,0.052223,0.090695,0.299894,0.754878
2022-06-30,0.201950,0.092259,0.014931,0.150120,0.034960,0.161880,0.203706,0.352302,0.227065,0.182362,...,0.259885,0.748619,0.282086,0.102717,0.449748,0.079015,0.070910,0.391695,0.138364,0.169210


In [32]:
print(RPS1.mean(axis=1).mean())

0.2113100572887138


In [33]:
print(RPS1.mean(axis=1))

date
2022-06-17    0.226413
2022-06-21    0.183714
2022-06-22    0.197438
2022-06-23    0.180147
2022-06-24    0.230207
2022-06-27    0.173660
2022-06-28    0.218154
2022-06-29    0.234587
2022-06-30    0.222627
2022-07-01    0.201480
2022-07-05    0.240855
2022-07-06    0.208054
2022-07-07    0.208073
2022-07-08    0.189195
2022-07-11    0.214484
2022-07-12    0.220204
2022-07-13    0.225724
2022-07-14    0.227885
2022-07-15    0.213739
2022-07-18    0.218521
2022-07-19    0.209058
2022-07-20    0.195667
2022-07-21    0.217759
2022-07-22    0.213796
dtype: float64


In [34]:
#save results 
RPS1.to_excel('26_June_22_July-gradient(baseline)-one_m_less.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('26_June_22_July-PROBABILITIES-gradient(baseline)-one_m_less.xlsx')

#####After feature importances
***

In [35]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [36]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-05-17    2
2022-05-18    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15189022 0.17815549 0.20110053 0.21579353 0.25306023]
Date
2022-05-17    1
2022-05-18    3
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.17981587 0.23205417 0.18150455 0.21092563 0.19569979]
Date
2022-05-17    4
2022-05-18    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                ABBV        AEP         AIZ        ALLE        AMAT         AMP  \
Date                                                                            
2022-06-21  147.75  92.066818  168.087662  103.014191  103.575127  244.206192   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-06-21  124.629997  196.050003  175.318207  150.179993  ...  94.955063   

             PYPL          RE       ROL       ROST         UNH         URI  \
Date                                                                         
2022-06-21  82.75  268.616547  36.46323  83.317162  520.825806  279.790009   

                    V        VRSK        XOM  
Date                                          
2022-06-21  215.80777  184.402573  86.905602  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-22  148.470001  93.614655  169.043716  102.944489  101.41066   

                   AMP        AMZN     AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-06-22  243.927475  122.419998  198.25  174.591156  153.009995  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-06-22  94.401566  81.050003  270.306335  36.373466  82.948456   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-06-22  519.779053  274.290009  213.322159  184.472458  86.24205  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                 

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-21  141.850006  104.940002  171.145065  105.991302  104.363106   

                  AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                  ...   
2022-07-21  284.73999  138.229996  214.979996  196.25322  162.240005  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-07-21  102.199997  96.559998  279.808899  36.540001  90.691246   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-07-21  546.604919  316.690002  212.820007  200.236176  94.080002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
D

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-22  140.339996  103.410004  164.163818  102.695572  100.552856   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-07-22  276.98999  133.220001  210.330002  193.584045  157.699997  ...   

                  PRU       PYPL         RE        ROL       ROST         UNH  \
Date                                                                            
2022-07-22  99.110001  93.580002  273.01001  36.400002  87.801399  542.866638   

                   URI           V        VRSK        XOM  
Date                                                       
2022-07-22  307.549988  208.610001  197.900055  94.010002  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-06-17,0.284809,0.123254,0.131944,0.266524,0.224599,0.127825,0.156041,0.076771,0.128344,0.079168,...,0.112772,0.294850,0.129429,0.086919,0.127886,0.068549,0.125790,0.116869,0.119170,0.279336
2022-06-21,0.095291,0.252606,0.110568,0.077409,0.206755,0.076454,0.193946,0.246986,0.065215,0.251643,...,0.112772,0.137537,0.225460,0.086919,0.085585,0.068549,0.176608,0.120414,0.062418,0.168383
2022-06-22,0.097545,0.243695,0.243525,0.117629,0.130283,0.076454,0.195516,0.119901,0.101605,0.134066,...,0.259094,0.202864,0.115688,0.086919,0.220007,0.205509,0.158511,0.074060,0.252342,0.164889
2022-06-23,0.194056,0.246948,0.114051,0.077156,0.130283,0.122726,0.193946,0.112127,0.101605,0.234087,...,0.112772,0.137537,0.070258,0.129392,0.223285,0.100932,0.107693,0.120414,0.119170,0.104629
2022-06-24,0.147674,0.262354,0.113607,0.074467,0.224599,0.242161,0.252759,0.112538,0.278570,0.079168,...,0.065687,0.294327,0.271919,0.241589,0.127886,0.145382,0.274552,0.245193,0.118014,0.104014
2022-06-27,0.287655,0.086852,0.107375,0.107212,0.148832,0.101816,0.099655,0.246890,0.296104,0.105030,...,0.057148,0.138098,0.273759,0.228131,0.085585,0.072148,0.270397,0.074060,0.120108,0.107569
2022-06-28,0.297942,0.086852,0.121204,0.128636,0.122471,0.232059,0.252759,0.072251,0.105132,0.133038,...,0.125663,0.134876,0.070388,0.242244,0.220007,0.145382,0.274566,0.120414,0.055409,0.164798
2022-06-29,0.287655,0.243889,0.103803,0.073957,0.150133,0.268839,0.154966,0.245312,0.066060,0.105030,...,0.065687,0.296232,0.274292,0.229218,0.244347,0.294305,0.112172,0.069731,0.109340,0.289354
2022-06-30,0.097545,0.081182,0.247558,0.220466,0.133999,0.133757,0.119398,0.245312,0.124877,0.112378,...,0.247325,0.296232,0.131746,0.129083,0.223285,0.098428,0.171711,0.247653,0.123006,0.103618


In [37]:
print(RPS2.mean(axis=1).mean())

0.15887937915273323


In [38]:
print(RPS2.mean(axis=1))

date
2022-06-17    0.169984
2022-06-21    0.149564
2022-06-22    0.152527
2022-06-23    0.151218
2022-06-24    0.167284
2022-06-27    0.161827
2022-06-28    0.163839
2022-06-29    0.164976
2022-06-30    0.162327
2022-07-01    0.166722
2022-07-05    0.164865
2022-07-06    0.148942
2022-07-07    0.156430
2022-07-08    0.147176
2022-07-11    0.168323
2022-07-12    0.163218
2022-07-13    0.166785
2022-07-14    0.169884
2022-07-15    0.164636
2022-07-18    0.146546
2022-07-19    0.155588
2022-07-20    0.147528
2022-07-21    0.145935
2022-07-22    0.156979
dtype: float64


In [41]:
#save results 
RPS2.to_excel('26_June_22_July-gradient(with_feature_imp)-one_m_less.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('26_June_22_July-PROBABILITIES-gradient(with_feature_imp)-one_m_less.xlsx')

####**For 24 July-19 August**

#####Features
***

In [42]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-07-01','2022-08-20') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-07-01  112.949997  114.550003  112.889999  114.260002  108.553429
 2021-07-02  114.519997  115.489998  113.889999  115.169998  109.417984
 2021-07-06  115.169998  115.900002  114.769997  115.730003  109.950020
 2021-07-07  115.910004  116.970001  115.309998  116.750000  110.919067
 2021-07-08  115.760002  116.809998  115.680000  116.349998  110.539055
 ...                ...         ...         ...         ...         ...
 2022-08-15  143.110001  143.279999  140.419998  142.289993  142.289993
 2022-08-16  142.110001  143.979996  142.080002  142.550003  142.550003
 2022-08-17  141.419998  142.759995  140.550003  141.440002  141.440002
 2022-08-18  141.490005  141.580002  139.820007  141.289993  141.289993
 2022-08-19  141.779999  143.360001  141.460007  141.850006  141.850006
 
 [287 rows x 5 columns],
 'AEP':                   Ope

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-07-24') & (stock['ABBV'].index <= '2022-08-19')]))

20


In [43]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-06-23']['Rank'])

Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    4.0
Name: Rank, dtype: float64
Date
2022-06-23    3.0
Name: Rank, dtype: float64
Date
2022-06-23    2.0
Name: Rank, dtype: float64
Date
2022-06-23    2.0
Name: Rank, dtype: float64
Date
2022-06-23    1.0
Name: Rank, dtype: float64
Date
2022-06-23    4.0
Name: Rank, dtype: float64
Date
2022-06-23    2.0
Name: Rank, dtype: float64
Date
2022-06-23    1.0
Name: Rank, dtype: float64
Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    4.0
Name: Rank, dtype: float64
Date
2022-06-23    3.0
Name: Rank, dtype: float64
Date
2022-06-23    3.0
Name: Rank, dtype: float64
Date
2022-06-23    4.0
Name: Rank, dtype: float64
Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    5.0
Name: Rank, dtype: float64
Date
2022-06-23    2.0
Name: Rank, dtype: float64


#####Gradient Boosting (baseline)
***

In [44]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-06-10    3.0
2022-06-13    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-14'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
[0.00761607 0.03192172 0.26028499 0.08982861 0.6103486 ]
Date
2022-06-10    4.0
2022-06-13    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-14'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
[0.01502975 0.10576276 0.01846183 0.69513104 0.16561461]
Date
2022-06-10    1.0
2022-06-13    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-14'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-15  142.289993  104.559998  166.573883  106.738075  109.499977   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-07-15  286.339996  143.179993  218.380005  200.366501  165.399994  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-07-15  103.603516  101.510002  273.487122  36.900002  89.844223   

                   UNH         URI           V        VRSK    XOM  
Date                                                               
2022-07-15  542.936401  337.519989  216.419998  205.327682  92.32  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-18  142.550003  104.459999  169.531708  107.992645  108.213264   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-07-18  289.059998  144.779999  219.240005  203.294617  165.350006  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-07-18  103.959343  102.080002  275.634125  37.490002  92.315536   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-07-18  544.411743  339.309998  217.139999  204.149658  91.459999  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-16  142.509995  100.190002  155.759995  93.150002  88.919998   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-08-16  277.350006  126.279999  201.320007  179.360001  156.130005  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-08-16  92.540001  96.400002  285.019989  35.799999  89.559998   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-08-16  522.909973  288.709991  195.369995  183.979996  94.830002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMA

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-07-14,0.274612,0.175018,0.085385,0.409889,0.199860,0.197858,0.166647,0.012995,0.263325,0.370410,...,0.051441,0.270884,0.048810,0.327800,0.194773,0.003557,0.285916,0.488435,0.321971,0.175182
2022-07-15,0.211441,0.109483,0.594799,0.403193,0.124151,0.402277,0.130667,0.147149,0.345187,0.274253,...,0.090752,0.412164,0.216535,0.054573,0.295903,0.079798,0.217035,0.172855,0.346339,0.576908
2022-07-18,0.121351,0.121986,0.370676,0.219403,0.086871,0.149276,0.634203,0.275150,0.188526,0.389702,...,0.089445,0.102249,0.138990,0.123813,0.121778,0.214265,0.227945,0.003603,0.039670,0.020711
2022-07-19,0.206152,0.244603,0.140680,0.046446,0.303815,0.319559,0.149048,0.465851,0.282096,0.195896,...,0.155997,0.212033,0.377154,0.060856,0.397726,0.099125,0.378671,0.129511,0.139658,0.039960
2022-07-20,0.271692,0.348650,0.013261,0.476981,0.188852,0.156691,0.149301,0.125096,0.155221,0.081098,...,0.150393,0.669756,0.102734,0.200018,0.168384,0.338718,0.234894,0.115844,0.186331,0.077970
2022-07-21,0.509514,0.236061,0.302762,0.108784,0.163313,0.097239,0.029233,0.058994,0.019967,0.245234,...,0.101850,0.029832,0.318625,0.194501,0.221684,0.363680,0.272775,0.105411,0.113640,0.248019
2022-07-22,0.164099,0.006928,0.095607,0.005306,0.533430,0.327530,0.703196,0.371240,0.093032,0.111045,...,0.195666,0.295714,0.096080,0.337849,0.380778,0.043176,0.106131,0.033404,0.307103,0.115564
2022-07-25,0.384969,0.077949,0.123127,0.002553,0.202712,0.109418,0.126873,0.326943,0.192909,0.113610,...,0.044723,0.353814,0.116140,0.321269,0.221694,0.274288,0.153604,0.171867,0.177848,0.090969
2022-07-26,0.002544,0.123426,0.006914,0.098119,0.204820,0.142689,0.302455,0.054529,0.190430,0.115289,...,0.129401,0.260165,0.063681,0.015065,0.125130,0.189347,0.208551,0.064361,0.428989,0.099882


In [45]:
print(RPS1.mean(axis=1).mean())

0.20588170957773952


In [46]:
print(RPS1.mean(axis=1))

date
2022-07-14    0.211569
2022-07-15    0.219476
2022-07-18    0.185170
2022-07-19    0.210437
2022-07-20    0.203687
2022-07-21    0.193971
2022-07-22    0.213332
2022-07-25    0.195605
2022-07-26    0.186288
2022-07-27    0.181197
2022-07-28    0.181705
2022-07-29    0.203337
2022-08-01    0.200223
2022-08-02    0.194106
2022-08-03    0.199707
2022-08-04    0.215078
2022-08-05    0.227539
2022-08-08    0.181006
2022-08-09    0.248359
2022-08-10    0.208512
2022-08-11    0.199402
2022-08-12    0.194384
2022-08-15    0.245388
2022-08-16    0.241684
dtype: float64


In [47]:
#save results 
#save results 
RPS1.to_excel('24_July_19_August-gradient(baseline)-one_m_less.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('24_July_19_August-PROBABILITIES-gradient(baseline)-one_m_less.xlsx')

#####After feature importances
***

In [48]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [49]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-06-10    3.0
2022-06-13    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-14'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
[0.14340192 0.20443803 0.17707515 0.20595782 0.26912707]
Date
2022-06-10    4.0
2022-06-13    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-14'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
[0.1609654  0.23094795 0.17088947 0.2298914  0.20730579]
Date
2022-06-10    1.0
2022-06-13    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-14'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-15  142.289993  104.559998  166.573883  106.738075  109.499977   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-07-15  286.339996  143.179993  218.380005  200.366501  165.399994  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-07-15  103.603516  101.510002  273.487122  36.900002  89.844223   

                   UNH         URI           V        VRSK    XOM  
Date                                                               
2022-07-15  542.936401  337.519989  216.419998  205.327682  92.32  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-18  142.550003  104.459999  169.531708  107.992645  108.213264   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-07-18  289.059998  144.779999  219.240005  203.294617  165.350006  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-07-18  103.959343  102.080002  275.634125  37.490002  92.315536   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-07-18  544.411743  339.309998  217.139999  204.149658  91.459999  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-16  142.509995  100.190002  155.759995  93.150002  88.919998   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-08-16  277.350006  126.279999  201.320007  179.360001  156.130005  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-08-16  92.540001  96.400002  285.019989  35.799999  89.559998   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-08-16  522.909973  288.709991  195.369995  183.979996  94.830002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMA

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-07-14,0.153256,0.104666,0.097634,0.129223,0.219592,0.255371,0.119484,0.071152,0.237574,0.111785,...,0.243582,0.206989,0.135014,0.132539,0.124781,0.075160,0.109243,0.250225,0.262469,0.271072
2022-07-15,0.153256,0.130940,0.252055,0.276608,0.213984,0.108134,0.150669,0.071152,0.274539,0.111785,...,0.068817,0.206989,0.110504,0.223362,0.241235,0.075160,0.109243,0.068817,0.246624,0.271072
2022-07-18,0.083100,0.104666,0.245207,0.141021,0.221883,0.108134,0.205227,0.257303,0.106304,0.108694,...,0.068817,0.136222,0.219820,0.132539,0.128742,0.100059,0.210113,0.119788,0.062212,0.176539
2022-07-19,0.147733,0.130940,0.122445,0.127876,0.209172,0.244396,0.104194,0.237237,0.235880,0.136074,...,0.123989,0.204125,0.219820,0.080949,0.217749,0.143216,0.197733,0.068666,0.063218,0.176539
2022-07-20,0.189445,0.220581,0.108845,0.220512,0.209177,0.125613,0.197382,0.071235,0.065933,0.105301,...,0.113288,0.136222,0.110504,0.129119,0.090880,0.206186,0.123304,0.068817,0.115104,0.107381
2022-07-21,0.100742,0.110383,0.277044,0.213618,0.209177,0.125613,0.205227,0.071235,0.123658,0.105301,...,0.113288,0.136222,0.135014,0.229748,0.215364,0.205933,0.119302,0.068817,0.120518,0.180370
2022-07-22,0.147151,0.081997,0.063505,0.103569,0.231457,0.108134,0.237465,0.236721,0.064633,0.134303,...,0.253065,0.137108,0.110504,0.248742,0.130871,0.286826,0.143653,0.119788,0.252097,0.180441
2022-07-25,0.289200,0.130940,0.106274,0.103069,0.214547,0.070239,0.103960,0.256059,0.106849,0.278750,...,0.068817,0.204125,0.111194,0.253287,0.250982,0.074772,0.108663,0.119788,0.113973,0.107553
2022-07-26,0.083100,0.130940,0.124366,0.073050,0.216290,0.108134,0.256056,0.118348,0.065833,0.134303,...,0.123989,0.295309,0.066584,0.253287,0.120620,0.145628,0.244157,0.068817,0.242094,0.163142


In [50]:
print(RPS2.mean(axis=1).mean())

0.15844373271845769


In [51]:
print(RPS2.mean(axis=1))

date
2022-07-14    0.168190
2022-07-15    0.165768
2022-07-18    0.145477
2022-07-19    0.158574
2022-07-20    0.144589
2022-07-21    0.145181
2022-07-22    0.163436
2022-07-25    0.163644
2022-07-26    0.168632
2022-07-27    0.148062
2022-07-28    0.156809
2022-07-29    0.159610
2022-08-01    0.164999
2022-08-02    0.150656
2022-08-03    0.157335
2022-08-04    0.158462
2022-08-05    0.166086
2022-08-08    0.161240
2022-08-09    0.162281
2022-08-10    0.163519
2022-08-11    0.145637
2022-08-12    0.161656
2022-08-15    0.162206
2022-08-16    0.160600
dtype: float64


In [52]:
#save results 
RPS2.to_excel('24_July_19_August-gradient(with_feature_imp)-real_data.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('24_July_19_August-PROBABILITIES-gradient(with_feature_imp)-real_data.xlsx')

####For the whole period with one month less training 
***

In [54]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-05-01','2022-08-20') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [55]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#####Gradient Boosting(baseline)
***

In [56]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-04-21    1.0
2022-04-22    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-23'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
[0.66712213 0.07885787 0.03896651 0.10577595 0.10927752]
Date
2022-04-21    1.0
2022-04-22    5.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-23'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
[0.12658503 0.08616359 0.32622446 0.44112742 0.01989951]
Date
2022-04-21    3.0
2022-04-22    5.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-23'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-24  150.928238  92.870499  172.150925  99.170822  97.111656   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-24  248.137924  116.459999  193.575409  164.681396  145.605087  ...   

                  PRU   PYPL         RE        ROL       ROST         UNH  \
Date                                                                        
2022-05-24  96.279518  77.68  281.48877  35.017067  77.258453  494.089691   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-24  248.289993  205.146637  172.702087  86.063782  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                      

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                                                                 
2022-05-25  151.720825  94.239746  172.300293  98.81237  97.361015   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-25  246.087448  113.220001  197.166199  166.274918  143.752029  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-25  95.824852  75.940002  281.329712  35.186615  76.072624   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-25  504.078339  251.990005  203.200089  173.810226  88.173286  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-16  142.509995  100.190002  155.759995  93.150002  88.919998   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-08-16  277.350006  126.279999  201.320007  179.360001  156.130005  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-08-16  92.540001  96.400002  285.019989  35.799999  89.559998   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-08-16  522.909973  288.709991  195.369995  183.979996  94.830002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMA

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-23,0.136535,0.336419,0.122381,0.086682,0.297802,0.024545,0.423271,0.438828,0.577518,0.110337,...,0.395250,0.539848,0.131827,0.221208,0.055986,0.451907,0.465595,0.065154,0.580059,0.338273
2022-05-24,0.136364,0.248479,0.099166,0.388710,0.205256,0.342986,0.038440,0.120487,0.178633,0.386333,...,0.057506,0.000079,0.026623,0.090838,0.385989,0.054510,0.308728,0.153660,0.000128,0.151256
2022-05-25,0.161735,0.242377,0.302538,0.000723,0.142082,0.139221,0.388243,0.086067,0.059309,0.222876,...,0.391023,0.136620,0.099781,0.177187,0.124324,0.143471,0.107653,0.078104,0.585256,0.132209
2022-05-26,0.154184,0.074655,0.336969,0.105522,0.048772,0.077470,0.389914,0.290910,0.197345,0.745272,...,0.169804,0.353813,0.164935,0.243125,0.160456,0.060265,0.231099,0.220039,0.174287,0.287173
2022-05-27,0.045936,0.258912,0.090168,0.071766,0.000445,0.359458,0.092104,0.114243,0.144459,0.199349,...,0.030405,0.233655,0.081458,0.045203,0.014301,0.085693,0.445681,0.116390,0.188357,0.157875
2022-05-31,0.092454,0.029631,0.030794,0.070112,0.349082,0.420037,0.686882,0.087062,0.116169,0.078603,...,0.123407,0.646063,0.059047,0.049706,0.556662,0.170477,0.291301,0.183531,0.154404,0.484502
2022-06-01,0.202723,0.313371,0.093167,0.159332,0.095849,0.075424,0.360043,0.228272,0.024017,0.026330,...,0.295161,0.591704,0.392093,0.040255,0.045198,0.201624,0.175165,0.173206,0.144240,0.097761
2022-06-02,0.311244,0.091948,0.317637,0.036140,0.104824,0.117700,0.267062,0.384996,0.284369,0.183063,...,0.103657,0.058980,0.013109,0.294819,0.101927,0.309909,0.349214,0.202122,0.003833,0.392037
2022-06-03,0.367077,0.431307,0.268208,0.267159,0.236854,0.200598,0.184241,0.255604,0.263265,0.187877,...,0.160208,0.524665,0.238325,0.090018,0.546700,0.297803,0.470107,0.101232,0.402349,0.172419


In [57]:
print(RPS1.mean(axis=1).mean())

0.20873601746966666


In [68]:
#for 24 June
print(RPS1.iloc[0].mean())
#for 22 July
print(RPS1.iloc[19].mean())
#for 19 August
print(RPS1.iloc[39].mean())

0.2428251863012746
0.19474809360781337
0.1942291172152514


In [59]:
RPS1.mean(axis=1)

date
2022-05-23    0.242825
2022-05-24    0.165279
2022-05-25    0.174258
2022-05-26    0.196016
2022-05-27    0.200564
2022-05-31    0.238834
2022-06-01    0.232199
2022-06-02    0.177384
2022-06-03    0.250339
2022-06-06    0.200468
2022-06-07    0.197889
2022-06-08    0.277915
2022-06-09    0.177335
2022-06-10    0.217699
2022-06-13    0.247097
2022-06-14    0.250617
2022-06-15    0.220618
2022-06-16    0.243268
2022-06-17    0.242043
2022-06-21    0.194748
2022-06-22    0.181874
2022-06-23    0.165306
2022-06-24    0.261816
2022-06-27    0.183520
2022-06-28    0.218519
2022-06-29    0.253224
2022-06-30    0.232662
2022-07-01    0.185850
2022-07-05    0.211691
2022-07-06    0.200609
2022-07-07    0.200762
2022-07-08    0.207667
2022-07-11    0.197338
2022-07-12    0.217027
2022-07-13    0.194321
2022-07-14    0.219848
2022-07-15    0.215418
2022-07-18    0.197823
2022-07-19    0.211804
2022-07-20    0.194229
2022-07-21    0.204340
2022-07-22    0.218546
2022-07-25    0.197413
2022-0

#####After feature importances
***

In [69]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [70]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-04-21    1.0
2022-04-22    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-23'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15454447 0.18537241 0.19484691 0.2267129  0.23852331]
Date
2022-04-21    1.0
2022-04-22    5.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-23'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16296018 0.22922596 0.18181892 0.22796103 0.1980339 ]
Date
2022-04-21    3.0
2022-04-22    5.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-23'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-24  150.928238  92.870499  172.150925  99.170822  97.111656   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-24  248.137924  116.459999  193.575409  164.681396  145.605087  ...   

                  PRU   PYPL         RE        ROL       ROST         UNH  \
Date                                                                        
2022-05-24  96.279518  77.68  281.48877  35.017067  77.258453  494.089691   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-24  248.289993  205.146637  172.702087  86.063782  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                      

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                                                                 
2022-05-25  151.720825  94.239746  172.300293  98.81237  97.361015   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-25  246.087448  113.220001  197.166199  166.274918  143.752029  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-25  95.824852  75.940002  281.329712  35.186615  76.072624   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-25  504.078339  251.990005  203.200089  173.810226  88.173286  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-16  142.509995  100.190002  155.759995  93.150002  88.919998   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-08-16  277.350006  126.279999  201.320007  179.360001  156.130005  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-08-16  92.540001  96.400002  285.019989  35.799999  89.559998   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-08-16  522.909973  288.709991  195.369995  183.979996  94.830002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMA

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-23,0.146586,0.259048,0.123710,0.072680,0.142133,0.097558,0.157668,0.247538,0.278692,0.074380,...,0.243046,0.289792,0.112524,0.085235,0.108400,0.283903,0.237252,0.125516,0.279140,0.162225
2022-05-24,0.106172,0.230646,0.059042,0.117354,0.119408,0.139008,0.189620,0.249081,0.129300,0.241510,...,0.118545,0.144094,0.072408,0.087295,0.223047,0.216665,0.236559,0.220990,0.098420,0.169339
2022-05-25,0.096459,0.230416,0.250232,0.116903,0.140878,0.073357,0.189620,0.113351,0.101963,0.074380,...,0.243046,0.144094,0.112524,0.135755,0.223047,0.216665,0.107833,0.113893,0.279140,0.183568
2022-05-26,0.096459,0.103324,0.124918,0.120657,0.215793,0.274789,0.275567,0.073425,0.129300,0.242625,...,0.118545,0.134850,0.132853,0.218107,0.109349,0.216665,0.208006,0.129728,0.249853,0.283286
2022-05-27,0.082828,0.226654,0.241829,0.071245,0.145177,0.139008,0.191832,0.073425,0.101963,0.131656,...,0.072124,0.144393,0.214671,0.109295,0.216292,0.067885,0.238421,0.111554,0.251555,0.269272
2022-05-31,0.149241,0.105718,0.056185,0.124169,0.227934,0.274789,0.277578,0.115060,0.067334,0.074185,...,0.119995,0.305421,0.072408,0.253565,0.134544,0.138231,0.128699,0.080855,0.054209,0.183568
2022-06-01,0.095305,0.264913,0.127535,0.120233,0.116907,0.139008,0.277578,0.118809,0.067334,0.074185,...,0.254295,0.304715,0.275297,0.087295,0.256281,0.283903,0.139756,0.129728,0.113791,0.269272
2022-06-02,0.284484,0.226654,0.104846,0.071245,0.142133,0.139008,0.160374,0.118809,0.278692,0.238849,...,0.123810,0.144094,0.214671,0.218107,0.110028,0.216665,0.209804,0.081051,0.249232,0.269272
2022-06-03,0.284769,0.268695,0.124918,0.113047,0.203700,0.097558,0.160374,0.247538,0.067334,0.131656,...,0.116585,0.206263,0.072408,0.109295,0.245350,0.284280,0.235650,0.080906,0.262569,0.183568


In [71]:
print(RPS2.mean(axis=1).mean())

0.15874028800355117


In [72]:
RPS2.mean(axis=1)


date
2022-05-23    0.167380
2022-05-24    0.149691
2022-05-25    0.151394
2022-05-26    0.158914
2022-05-27    0.154487
2022-05-31    0.160740
2022-06-01    0.169500
2022-06-02    0.155366
2022-06-03    0.161193
2022-06-06    0.152655
2022-06-07    0.151419
2022-06-08    0.160766
2022-06-09    0.159755
2022-06-10    0.162740
2022-06-13    0.164847
2022-06-14    0.163143
2022-06-15    0.164915
2022-06-16    0.166070
2022-06-17    0.168336
2022-06-21    0.149965
2022-06-22    0.151115
2022-06-23    0.151692
2022-06-24    0.167384
2022-06-27    0.160980
2022-06-28    0.161649
2022-06-29    0.165374
2022-06-30    0.162889
2022-07-01    0.165030
2022-07-05    0.163800
2022-07-06    0.150191
2022-07-07    0.157101
2022-07-08    0.147660
2022-07-11    0.165719
2022-07-12    0.158662
2022-07-13    0.164776
2022-07-14    0.167306
2022-07-15    0.162055
2022-07-18    0.147272
2022-07-19    0.154877
2022-07-20    0.148848
2022-07-21    0.147909
2022-07-22    0.159688
2022-07-25    0.163122
2022-0

In [ ]:
#for 24 June
print(RPS2.iloc[0].mean())
#for 22 July
print(RPS2.iloc[19].mean())
#for 19 August
print(RPS2.iloc[39].mean())

0.16580516236896573
0.1510132733344073
0.14958952521542782


In [73]:
#SAVE FINAL RESULTS
RPS2.to_excel('Q2-submissions-running-for-the-whole-period.xlsx')